# Metody Kryptografii w analizie danych

# Kryptosystemy częściowo homomorficzne

## RSA

Mamy klucz publiczny $(n,e)$ i klucz prywatny $(n,d)$. Szyfrujemy tym samym kluczem publicznym dwie wiadomości $m_1$ i $m_2$. Otrzymane szyfrogramy są postaci:
$$c_1=m_1^e\mod n\qquad c_2=m_2^e\mod n.$$

Jeżeli teraz spróbujemy zdeszyfrować kluczem prywatnym iloczyn otrzymanych szyfrogramów, to otrzymamy
$$(c_1c_2)^d\mod n=c_1^dc_2^d\mod n=(c_1^d\mod n)(c_2^d\mod n)=m_1m_2$$

Otrzymaliśmy zatem, że
$$D\big(E(m_1)E(m_2)\big)=m_1m_2$$
czyli **RSA jest częściowo homomorficzny ze względu na mnożenie**.

## Ćwiczenie 1.

Korzystając z implementacji RSA z ostatnich zajęć sprawdź czy są limity dla liczby homomorficznych operacji mnożenia (tzn. czy od jakiejś liczby operacji na szyfrogramach zaczynamy otrzymywać błędne deszyfrowanie).

In [4]:
# p = 77777677777
# q = 99999199999 
# p=101
# q=103

def generate_keys_rsa():
    p = 77777677777
    q = 99999199999 
    n = p*q
    eu = (p-1)*(q-1)
    e = -1
    for i in range(10,eu-1):
        if coprime(eu,i):
            e = i
            break
    d = e**(-1)%eu
    return n, e, d

def encrypt_rsa(m, n, e):
    return (m**e)%n

def decipher_rsa(c, d):
    return c**d

def gcd(p, q):
    while q != 0:
        p, q = q, p % q
    return p

def coprime(a, b):
    return gcd(a, b) == 1

n_rsa,e_rsa,d_rsa = generate_keys_rsa()
m1 = 9
m2 = 3

c1 = encrypt_rsa(m1, n_rsa, e_rsa)
c2 = encrypt_rsa(m2, n_rsa, e_rsa)
print(f"c1 {c1},c2 {c2}, d {d_rsa}, e {e_rsa}, n {n_rsa}")
m1_deciphered = decipher_rsa(c1,d_rsa)
m2_deciphered = decipher_rsa(c2,d_rsa)

print("m1 ",m1_deciphered)
print("m2 ",m2_deciphered)

print(decipher_rsa(c1*c2,d_rsa))
print(m1*m2)


c1 31381059609,c2 177147, d 0.09090909090909091, e 11, n 7777705555480000722223
m1  9.0
m2  3.0
27.000000000000004
27


Dla p = 77777677777 i q = 99999199999 limit to 97. Dla p = 10301 i q = 10501 limit to 5.

## Kryptosystem Pailliera

### Generowanie kluczy
- wybieramy losowo liczby pierwsze $p$, $q$ o tej samej długości w zapisie dziesiętnym i obliczamy $n=pq$, $g=n+1$, $\lambda=\phi(n)=(p-1)(q-1)$ oraz  $\mu =\phi (n)^{-1}{\mod {n}}$.
- kluczem publicznym jest $(n,g)$ a kluczem prywatnym $(\lambda,\mu)$


### Szyfrowanie
Szyfrujemy liczbę $0\leq m<n$. Wybieramy losowe $r<n$ względnie pierwsze z $n$ i obliczamy szyfrogram $$c=g^{m}\cdot r^{n}{\mod {n}}^{2}.$$


### Deszyfrowanie
Obliczamy $$m=\mu L(c^{\lambda }{\bmod {n}}^{2}){\bmod {n}},$$ gdzie  $L(x)$ to największa liczba naturalna $\nu$ taka, że $x-1\geq \nu n$.

Pomimo czynnika losowego przy szyfrowaniu, kryptosystem Pailliera ma własność $$D\big(E(m_{1},r_{1})\cdot E(m_{2},r_{2})\big)=m_{1}+m_{2}$$
a zatem jest **częściowo homomorficzny ze względu na dodawanie**.

## Ćwiczenie 2.

Zaimplementuj kryptosystem Pailliera. Sprawdź, czy są limity homomorficznych operacji dodawania.

In [3]:
from random import randint
# p = 77777677777
# q = 99999199999 
# p = 10301
# q = 10501



def generate_keys_paillier():
    p=101
    q=103
    n = p*q
    g=n+1
    l = (p-1)*(q-1)
    u = pow(l, -1, n)
    r_canditates = []
    for i in range(2,n):
        if coprime(n,i):
            r_canditates.append(i)
        if len(r_canditates) == 20:
            break
    
    r1 = r_canditates[randint(0,len(r_canditates)-1)]
    r2 = r_canditates[randint(0,len(r_canditates)-1)]
        
    return n, u, g, r1, r2, l


def encrypt_paillier(m, g, n, r):
    return (g**m)*(r**n)%(n**2)

def L(x,n):
    return (x-1)//n
    

def decipher_paillier(c, u, n, l):
    return u*L(c**l%(n**2),n)%n

n_paillier,u_paillier,g_paillier, r1_paillier, r2_paillier, l_paillier = generate_keys_paillier()
print(f"n {n_paillier}, u {u_paillier}, g {g_paillier}, r {r1_paillier}, r2 {r2_paillier}, l {l_paillier}")

m1 = n_paillier//2
m2 = n_paillier//2+1


c1 = encrypt_paillier(m1, g_paillier, n_paillier, r1_paillier)
c2 = encrypt_paillier(m2, g_paillier, n_paillier, r2_paillier)
print(f"c1 {c1}")
print(f"c2 {c2}")

m1_deciphered = decipher_paillier(c1,u_paillier,n_paillier,l_paillier)
m2_deciphered = decipher_paillier(c2,u_paillier,n_paillier,l_paillier)
print(f"m1_deciphered {m1_deciphered}")
print(f"m2_deciphered {m2_deciphered}")


n 10403, u 6867, g 10404, r 19, r2 16, l 10200
c1 63918851
c2 69387965
m1_deciphered 5201
m2_deciphered 5202


In [20]:
encrypt_paillier(6, g_paillier, n_paillier, r1_paillier)

92204608

In [ ]:
print(f"m1 + m2 = {m1 + m2}")
print(f"decipher (c1*c2) = {decipher_paillier(c1*c2, u_paillier, n_paillier, l_paillier)}")

m1 + m2 = 10403
decipher (c1*c2) = 0


Limit jest gdy suma m1 i m2 > n

Kryptosystem nazywamy *w pewnym sensie homomorficznym*, jeżeli operacjami na samych szyfrogramach $E(m_1),...,E(m_k)$ jesteśmy w stanie obliczyć pewne określone kombinacje dodawania i mnożenia oryginalnych wiadomości $m_1,...,m_k$.

## Ćwiczenie 3.

Wykorzystując RSA i Pailliera zaimplementuj kryptosystem, który będzie w stanie obliczyć $(m_1m_2+m_3)\cdot m_4$. Wszystkie działania muszą się odbywać na danych zaszyfrowanych (nie możemy ujawnić chmurze żadnej wiadomości $m_1$, $m_2$, $m_3$, $m_4$). Czy jesteśmy w stanie w ten sposób skonstruować kryptosystem w pewnym sensie homomorficzny?

In [31]:
# TYPE YOUR CODE BELOW
m1 = 2
m2 = 3
m3 = 4
m4 = 5

expected_result = ((m1*m2+m3)*m4)
print(expected_result)

def encrypt4(m1, m2, m3, m4):
    c1 = encrypt_rsa(m1, n_rsa, e_rsa)
    c2 = encrypt_rsa(m2, n_rsa, e_rsa)
    c3 = encrypt_paillier(m3, g_paillier, n_paillier, r1_paillier)
    c4 = encrypt_rsa(m4, n_rsa, e_rsa)
    return c1, c2, c3, c4

c1, c2, c3, c4 = encrypt4(m1, m2, m3, m4)

rsa_m1m2 = c1*c2
# print(decipher_rsa(rsa_m1m2, d_rsa))
p_m1m2 = encrypt_paillier(round(decipher_rsa(rsa_m1m2, d_rsa)),g_paillier,n_paillier,r1_paillier)
p_m1m2m3 = p_m1m2*c3
r_m1m2m3m4 = encrypt_rsa(round(decipher_paillier(p_m1m2m3, u_paillier,n_paillier, l_paillier)), n_rsa, e_rsa)*c4


print(round(decipher_rsa(r_m1m2m3m4,d_rsa)))


50
50
